<a href="https://colab.research.google.com/github/wigglytuff-tu/Analytics-Coords/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os

import torch
import torchvision
from torch.optim import Adam
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [5]:
!kaggle datasets download -d itsahmad/indoor-scenes-cvpr-2019

100% 2.33G/2.34G [00:32<00:00, 101MB/s] 
100% 2.34G/2.34G [00:32<00:00, 76.2MB/s]


In [ ]:
!ls
!unzip \*.zip  && rm *.zip

In [8]:
!ls

indoorCVPR_09		  indoor-scenes-cvpr-2019.zip  TestImages.txt
indoorCVPR_09annotations  kaggle.json		       TrainImages.txt


In [ ]:
data_dir  = './indoorCVPR_09/Images'
classes = os.listdir(data_dir)
print(classes)
len(classes)

In [10]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

transformations = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor(),transforms.RandomHorizontalFlip(),
                                      transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5, 0.5, 0.5])])

dataset = ImageFolder(data_dir, transform = transformations)

In [14]:
import matplotlib.pyplot as plt   
%matplotlib inline

def show_sample(img, label):
    print("Label:", dataset.classes[label], "(Class No: "+ str(label) + ")")
    plt.imshow(img.permute(1, 2, 0))

train_ds, val_ds, test_ds = torch.utils.data.random_split(dataset, [13500, 1500, 620])
len(train_ds), len(val_ds), len(test_ds)

(13500, 1500, 620)

In [34]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout2d(0.2)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout2d(0.3)
        )

        self.conv5 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128)
        )
        self.conv6 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout2d(0.4)
        )
        self.conv7 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256)
        )
        self.conv8 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout2d(0.4)
        )

        self.fc = nn.Linear(256, 67)  # number of classes = 67

    def forward(self, x):
        x = self.conv8(self.conv7(self.conv6(self.conv5(self.conv4(self.conv3(self.conv2(self.conv1(x))))))))
        
        x = F.avg_pool2d(x, kernel_size=x.shape[2:])
        x = x.view(x.shape[0], -1)

        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [46]:
def train(model, dataloader, epoch, optimizer, criterion):
         
      model.train()
      train_loss = []
      train_acc = []
      for batch_idx, (data, target) in enumerate(dataloader):
          data, target = data.cuda(), target.cuda()
          output = model(data)
          
          optimizer.zero_grad()    
          loss = criterion(output, target)
          loss.backward()
          optimizer.step()
          
          pred_cls = output.max(1)[1]
          correct = pred_cls.eq(target.long().data).cpu().sum()
          
          train_acc.append(correct.item()/data.shape[0])
          train_loss.append(loss.item())
          
          if batch_idx % 1 == 0:
              print('Train Epoch: {} [({:.0f}%)]\tTrain_Loss: {:.4f}\tTrain_Acc: {:.4f}%'.format(
              epoch+1, 100. * batch_idx / len(dataloader), np.mean(train_loss), 100*np.mean(train_acc)))
      print()

In [47]:
def evaluate_model(model, testdataloader):

      val_acc = []
      val_loss = []
      model.eval()
      
      with torch.no_grad():
          for data, target in testdataloader:
              data, target = data.cuda(), target.cuda()
              output = model(data)
              pred_cls = output.max(1)[1]
              correct = pred_cls.eq(target.long().data).cpu().sum()
              val_acc.append(correct.item()/data.shape[0])
              
      print("Val Acc: {:.4f}%".format(100*np.mean(val_acc)))
      return np.mean(val_acc)    

In [50]:
def main():
    
   # Defining the dataloader
    
    train_dl = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle = True)
    
    val_dl = torch.utils.data.DataLoader(val_ds, batch_size=32, shuffle = False)
    
    # defining the model
    model = CNN()
    # defining the optimizer
    optimizer = Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    # defining the loss function
    criterion = nn.CrossEntropyLoss()

    if torch.cuda.is_available():
        model = model.cuda()
        criterion = criterion.cuda()
    
    print("Structure of the Model",model)
    
    #Training the model 
    n_epochs = 25
    best = 0
    for epoch in range(n_epochs):
        train(model, train_dl, epoch, optimizer, criterion)
        val_acc = evaluate_model(model, val_dl)
        if val_acc > best:
            best = val_acc
            #torch.save(model.state_dict(), "best.ckpt")
    
    print("Training Finished")

In [ ]:
main()

Train Epoch: 3 [(91%)]	Train_Loss: 3.3236	Train_Acc: 15.3564%
Train Epoch: 3 [(91%)]	Train_Loss: 3.3224	Train_Acc: 15.3815%
Train Epoch: 3 [(91%)]	Train_Loss: 3.3234	Train_Acc: 15.3578%
Train Epoch: 3 [(91%)]	Train_Loss: 3.3228	Train_Acc: 15.3666%
Train Epoch: 3 [(92%)]	Train_Loss: 3.3227	Train_Acc: 15.3592%
Train Epoch: 3 [(92%)]	Train_Loss: 3.3226	Train_Acc: 15.3358%
Train Epoch: 3 [(92%)]	Train_Loss: 3.3213	Train_Acc: 15.3846%
Train Epoch: 3 [(92%)]	Train_Loss: 3.3220	Train_Acc: 15.3772%
Train Epoch: 3 [(93%)]	Train_Loss: 3.3211	Train_Acc: 15.3699%
Train Epoch: 3 [(93%)]	Train_Loss: 3.3206	Train_Acc: 15.3785%
Train Epoch: 3 [(93%)]	Train_Loss: 3.3203	Train_Acc: 15.3871%
Train Epoch: 3 [(93%)]	Train_Loss: 3.3199	Train_Acc: 15.3877%
Train Epoch: 3 [(94%)]	Train_Loss: 3.3206	Train_Acc: 15.3725%
Train Epoch: 3 [(94%)]	Train_Loss: 3.3211	Train_Acc: 15.3652%
Train Epoch: 3 [(94%)]	Train_Loss: 3.3212	Train_Acc: 15.3580%
Train Epoch: 3 [(94%)]	Train_Loss: 3.3226	Train_Acc: 15.3274%
Train Ep